In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)

import os
for dirname, _, filenames in os.walk('C:/xampp/htdocs/SE J COMP'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

C:/xampp/htdocs/SE J COMP\connection.php
C:/xampp/htdocs/SE J COMP\failure.html
C:/xampp/htdocs/SE J COMP\first_page.html
C:/xampp/htdocs/SE J COMP\input.csv
C:/xampp/htdocs/SE J COMP\login.css
C:/xampp/htdocs/SE J COMP\login.html
C:/xampp/htdocs/SE J COMP\readoutput.php
C:/xampp/htdocs/SE J COMP\register.html
C:/xampp/htdocs/SE J COMP\SE J COMP - Shortcut.lnk
C:/xampp/htdocs/SE J COMP\second_page.html
C:/xampp/htdocs/SE J COMP\support.html
C:/xampp/htdocs/SE J COMP\upload.php
C:/xampp/htdocs/SE J COMP\validate.php


In [2]:
data = pd.read_csv('C:/xampp/htdocs/SE J COMP/input.csv')
data.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [4]:
categorical_features = ['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score', 'Var_1']
numerical_features = ['Age', 'Work_Experience', 'Family_Size']

to_drop = ['ID'] # contain unique values

In [5]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import os
import joblib

In [6]:
df = data.copy()
path = '/working'
for i, feature in enumerate(categorical_features):
    le = LabelEncoder()

    # create directory to save label encoding models
    if not os.path.exists(os.path.join(path, "TextEncoding")):
        os.makedirs(os.path.join(path, "TextEncoding"))

    # perform label encoding
    le.fit(df[feature])
    
    # save the encoder
    joblib.dump(le, open(os.path.join(path, "TextEncoding/le_{}.sav".format(feature)), 'wb'))
    
    # transfrom training data
    df[feature] = le.transform(df[feature])

    # get classes & remove first column to elude from dummy variable trap
    columns = list(map(lambda x: feature+' '+str(x), list(le.classes_)))[1:]
    
    # save classes
    joblib.dump(columns, 
                open(os.path.join(path, "TextEncoding/le_{}_classes.sav".format(feature)), 'wb'))

In [7]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [8]:
# Calculating VIF
vif = pd.DataFrame()
temp = df.dropna()
vif["variables"] = [feature for feature in categorical_features+numerical_features if feature not in ['Age', 'Var_1']]
vif["VIF"] = [variance_inflation_factor(temp[vif['variables']].values, i) for i in range(len(vif["variables"]))]
print(vif)

         variables       VIF
0           Gender  2.172928
1     Ever_Married  2.547924
2        Graduated  2.526486
3       Profession  2.365052
4   Spending_Score  3.231881
5  Work_Experience  1.546193
6      Family_Size  3.222219


In [9]:
missingValueFeatures = pd.DataFrame({'missing %': data.isnull().sum()*100/len(data)})
missingValueFeatures[missingValueFeatures['missing %']>0]

,missing %
Ever_Married,1.735250
Graduated,0.966782
Profession,1.536936
Work_Experience,10.275161
Family_Size,4.152206
Var_1,0.941993


In [10]:
data_missing = data.dropna().reset_index(drop=True)

In [11]:
# Percentage of outliers present in each variable
outlier_percentage = {}
for feature in numerical_features:
    tempData = data_missing.sort_values(by=feature)[feature]
    Q1, Q3 = tempData.quantile([0.25, 0.75])
    IQR = Q3 - Q1
    Lower_range = Q1 - (1.5 * IQR)
    Upper_range = Q3 + (1.5 * IQR)
    outlier_percentage[feature] = round((((tempData<(Q1 - 1.5 * IQR)) | (tempData>(Q3 + 1.5 * IQR))).sum()/tempData.shape[0])*100,2)
outlier_percentage

{'Age': 1.14, 'Work_Experience': 2.64, 'Family_Size': 1.2}

In [12]:
df = data_missing.copy()
path = '/working'
for i, feature in enumerate(categorical_features):
    
    le = LabelEncoder()
    ohe = OneHotEncoder(sparse=False)

    # create directory to save label encoding models
    if not os.path.exists(os.path.join(path, "TextEncoding")):
        os.makedirs(os.path.join(path, "TextEncoding"))

    # perform label encoding
    le.fit(df[feature])
    # save the encoder
    joblib.dump(le, open(os.path.join(path, "TextEncoding/le_{}.sav".format(feature)), 'wb'))
    
    # transfrom training data
    df[feature] = le.transform(df[feature])

    # get classes & remove first column to elude from dummy variable trap
    columns = list(map(lambda x: feature+' '+str(x), list(le.classes_)))[1:]
    
    # save classes
    joblib.dump(columns, 
                open(os.path.join(path, "TextEncoding/le_{}_classes.sav".format(feature)), 'wb'))
    # load classes
    columns = joblib.load(
        open(os.path.join(path, "TextEncoding/le_{}_classes.sav".format(feature)), 'rb'))

    if len(le.classes_)>2:
        # perform one hot encoding
        ohe.fit(df[[feature]])
        # save the encoder
        joblib.dump(ohe, open(os.path.join(path, "TextEncoding/ohe_{}.sav".format(feature)), 'wb'))

        # transfrom training data
        # removing first column of encoded data to elude from dummy variable trap
        tempData = ohe.transform(df[[feature]])[:, 1:]

        # create Dataframe with columns as classes
        tempData = pd.DataFrame(tempData, columns=columns)
    else:
        tempData = df[[feature]]
    
    # create dataframe with all the label encoded categorical features along with hot encoding
    if i==0:
        encodedData = pd.DataFrame(data=tempData, columns=tempData.columns.values.tolist())
    else:
        encodedData = pd.concat([encodedData, tempData], axis=1)

In [13]:
# merge numerical features and categorical encoded features
df = df[numerical_features+['Segmentation']]
df = pd.concat([df, encodedData], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       6665 non-null   int64  
 1   Work_Experience           6665 non-null   float64
 2   Family_Size               6665 non-null   float64
 3   Segmentation              6665 non-null   object 
 4   Gender                    6665 non-null   int32  
 5   Ever_Married              6665 non-null   int32  
 6   Graduated                 6665 non-null   int32  
 7   Profession Doctor         6665 non-null   float64
 8   Profession Engineer       6665 non-null   float64
 9   Profession Entertainment  6665 non-null   float64
 10  Profession Executive      6665 non-null   float64
 11  Profession Healthcare     6665 non-null   float64
 12  Profession Homemaker      6665 non-null   float64
 13  Profession Lawyer         6665 non-null   float64
 14  Professi

In [14]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import metrics, preprocessing
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from xgboost import XGBClassifier

In [15]:
train_data = df.copy()
feature_cols = [feature for feature in train_data.columns if feature not in(['Segmentation'])]
print('features used- ', feature_cols)

''' Rescaling to [0,1] '''
scaler = StandardScaler()
scaler.fit(train_data[feature_cols])
train_data[feature_cols] = scaler.transform(train_data[feature_cols])

features used-  ['Age', 'Work_Experience', 'Family_Size', 'Gender', 'Ever_Married', 'Graduated', 'Profession Doctor', 'Profession Engineer', 'Profession Entertainment', 'Profession Executive', 'Profession Healthcare', 'Profession Homemaker', 'Profession Lawyer', 'Profession Marketing', 'Spending_Score High', 'Spending_Score Low', 'Var_1 Cat_2', 'Var_1 Cat_3', 'Var_1 Cat_4', 'Var_1 Cat_5', 'Var_1 Cat_6', 'Var_1 Cat_7']


In [16]:
train_data['Segmentation'] = train_data['Segmentation'].map({'A':0, 'B':1, 'C':2, 'D':3})
X = train_data[feature_cols]
y = train_data['Segmentation']

validation_size = 0.50
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=validation_size, 
                                                    random_state=0, stratify=y)

In [17]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [18]:
y_pred = model.predict(X_train)

print('Train metrics...')
print(confusion_matrix(y_train, y_pred))
print(classification_report(y_train, y_pred))

y_pred = model.predict(X_test)

print('Validation metrics...')
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Train metrics...
[[397 107 166 138]
 [215 193 287  91]
 [114 106 534 106]
 [167  45  45 621]]
              precision    recall  f1-score   support

           0       0.44      0.49      0.47       808
           1       0.43      0.25      0.31       786
           2       0.52      0.62      0.56       860
           3       0.65      0.71      0.68       878

    accuracy                           0.52      3332
   macro avg       0.51      0.52      0.51      3332
weighted avg       0.51      0.52      0.51      3332

Validation metrics...
[[406 106 144 152]
 [251 152 295  88]
 [103  96 555 106]
 [190  37  48 604]]
              precision    recall  f1-score   support

           0       0.43      0.50      0.46       808
           1       0.39      0.19      0.26       786
           2       0.53      0.65      0.58       860
           3       0.64      0.69      0.66       879

    accuracy                           0.52      3333
   macro avg       0.50      0.51      0.49   

In [19]:
''' metrics on original data '''
y_pred = model.predict(train_data[feature_cols])

# def make_cm(matrix, columns):
#     n = len(columns)
#     act = ['actual Segmentation'] * n
#     pred = ['predicted Segmentation'] * n

#     cm = pd.DataFrame(matrix, 
#         columns=[pred, columns], index=[act, columns])
#     return cm

# df_matrix=make_cm(
#     confusion_matrix(train_data['Segmentation'], y_pred),['A', 'B', 'C', 'D'])

# display(df_matrix)
# print(classification_report(train_data['Segmentation'], y_pred))
y_pred
a = 0
b = 0
c = 0
d = 0
for i in y_pred:
    if i == 0:
        a = a+1
    elif i == 1:
        b = b+1
    elif i == 2:
        c = c+1
    elif i == 3:
        d = d+1
a = str(a)
b = str(b)
c = str(c)
d = str(d)

lst = [a, b, c, d]
lst
file = open('output.csv', 'w+', newline = '')
with file:
    write = csv.writer(file)
    write.writerow(lst)